In [36]:
!pip install tabpfn catboost autogluon lightgbm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
df=pd.read_csv("/kaggle/input/cybersecurity-intrusion-detection-dataset/cybersecurity_intrusion_data.csv")

In [4]:
df.nunique()

session_id             9537
network_packet_size     959
protocol_type             3
login_attempts           13
session_duration       9532
encryption_used           2
ip_reputation_score    9537
failed_logins             6
browser_type              5
unusual_time_access       2
attack_detected           2
dtype: int64

In [5]:
df.sample()

,session_id,network_packet_size,protocol_type,login_attempts,session_duration,encryption_used,ip_reputation_score,failed_logins,browser_type,unusual_time_access,attack_detected
2962,SID_02963,394,UDP,5,7.159503,NaN,0.149156,2,Unknown,0,1


In [6]:
df.isnull().sum()

session_id                0
network_packet_size       0
protocol_type             0
login_attempts            0
session_duration          0
encryption_used        1966
ip_reputation_score       0
failed_logins             0
browser_type              0
unusual_time_access       0
attack_detected           0
dtype: int64

In [7]:
len(df)

9537

In [8]:
df=df.dropna()
df

,session_id,network_packet_size,protocol_type,login_attempts,session_duration,encryption_used,ip_reputation_score,failed_logins,browser_type,unusual_time_access,attack_detected
0,SID_00001,599,TCP,4,492.983263,DES,0.606818,1,Edge,0,1
1,SID_00002,472,TCP,3,1557.996461,DES,0.301569,0,Firefox,0,0
2,SID_00003,629,TCP,3,75.044262,DES,0.739164,2,Chrome,0,1
3,SID_00004,804,UDP,4,601.248835,DES,0.123267,0,Unknown,0,1
4,SID_00005,453,TCP,5,532.540888,AES,0.054874,1,Firefox,0,0
...,...,...,...,...,...,...,...,...,...,...,...
9528,SID_09529,535,TCP,7,50.518476,DES,0.767659,1,Edge,1,1
9531,SID_09532,746,TCP,7,315.151758,DES,0.190059,3,Chrome,0,1
9532,SID_09533,194,ICMP,3,226.049889,AES,0.517737,3,Chrome,0,1
9534,SID_09535,664,TCP,5,35.170248,AES,0.359200,1,Firefox,0,0


In [11]:
label_encoder=LabelEncoder()
df["protocol_type"]=label_encoder.fit_transform(df["protocol_type"])
df["encryption_used"]=label_encoder.fit_transform(df["encryption_used"])
df["browser_type"]=label_encoder.fit_transform(df["browser_type"])

# df["browser_type"]

<ipython-input-11-b5d3f0d49ac4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["protocol_type"]=label_encoder.fit_transform(df["protocol_type"])
<ipython-input-11-b5d3f0d49ac4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["encryption_used"]=label_encoder.fit_transform(df["encryption_used"])
<ipython-input-11-b5d3f0d49ac4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [13]:
df

,session_id,network_packet_size,protocol_type,login_attempts,session_duration,encryption_used,ip_reputation_score,failed_logins,browser_type,unusual_time_access,attack_detected
0,SID_00001,599,1,4,492.983263,1,0.606818,1,1,0,1
1,SID_00002,472,1,3,1557.996461,1,0.301569,0,2,0,0
2,SID_00003,629,1,3,75.044262,1,0.739164,2,0,0,1
3,SID_00004,804,2,4,601.248835,1,0.123267,0,4,0,1
4,SID_00005,453,1,5,532.540888,0,0.054874,1,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...
9528,SID_09529,535,1,7,50.518476,1,0.767659,1,1,1,1
9531,SID_09532,746,1,7,315.151758,1,0.190059,3,0,0,1
9532,SID_09533,194,0,3,226.049889,0,0.517737,3,0,0,1
9534,SID_09535,664,1,5,35.170248,0,0.359200,1,2,0,0


In [14]:
df=df.drop(columns=["session_id"])

In [15]:
X=df.drop(columns=["attack_detected"])
Y=df["attack_detected"]

In [17]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.1,random_state=42,shuffle=True)

In [18]:
X_train.shape

(6813, 9)

In [20]:
random_classif=RandomForestClassifier()
random_classif.fit(X_train,Y_train)
random_classif.score(X_test,Y_test)

0.8905013192612137

# # pretrained model# 

In [23]:
from tabpfn import TabPFNClassifier
tabpfn_classif=RandomForestClassifier()
tabpfn_classif.fit(X_train,Y_train)
tabpfn_classif.score(X_test,Y_test)

0.8931398416886543

In [30]:
predict_tabpfin=tabpfn_classif.predict(X_test.iloc[:5,:])
print("prediciton of tabpfin",predict_tabpfin)
print("prediciton of tabpfin",Y_train[:5])
# Y_train[:5]

prediciton of tabpfin [0 0 1 0 1]
prediciton of tabpfin 8176    1
7378    0
2414    1
3490    1
3635    0
Name: attack_detected, dtype: int64


In [24]:
Y_train.unique(
    
)

array([1, 0])

In [34]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(iterations=100, task_type="CPU")  
model.fit(X_train, Y_train)  # Needs training
model.score(X_test,Y_test)
# preds = model.predict(X_test)  
# predict_tabpfin=tabpfn_classif.predict(X_test.iloc[:5,:])
# print("prediciton of tabpfin",predict_tabpfin)
# print("prediciton of tabpfin",Y_train[:5])
# Y_train[:5]


Learning rate set to 0.193094
0:	learn: 0.5937831	total: 58.1ms	remaining: 5.75s
1:	learn: 0.5223117	total: 59.9ms	remaining: 2.93s
2:	learn: 0.4696569	total: 61.2ms	remaining: 1.98s
3:	learn: 0.4332811	total: 62.8ms	remaining: 1.51s
4:	learn: 0.4074641	total: 65ms	remaining: 1.24s
5:	learn: 0.3849852	total: 66.8ms	remaining: 1.05s
6:	learn: 0.3682556	total: 68.6ms	remaining: 912ms
7:	learn: 0.3541042	total: 70.2ms	remaining: 807ms
8:	learn: 0.3422606	total: 71.7ms	remaining: 725ms
9:	learn: 0.3350085	total: 73.4ms	remaining: 661ms
10:	learn: 0.3270335	total: 74.7ms	remaining: 604ms
11:	learn: 0.3216682	total: 76ms	remaining: 558ms
12:	learn: 0.3164884	total: 78.1ms	remaining: 522ms
13:	learn: 0.3139419	total: 79.4ms	remaining: 488ms
14:	learn: 0.3111718	total: 80.7ms	remaining: 457ms
15:	learn: 0.3089116	total: 82.2ms	remaining: 431ms
16:	learn: 0.3067885	total: 83.6ms	remaining: 408ms
17:	learn: 0.3041904	total: 85.1ms	remaining: 388ms
18:	learn: 0.3028539	total: 87.3ms	remaining: 37

0.8931398416886543

In [ ]:
!pip install --upgrade scikit-learn


In [42]:
from autogluon.tabular import TabularPredictor

predictor = TabularPredictor(label="labels").fit(train_data=X_train)
preds = predictor.predict(X_test)


ImportError: cannot import name '_is_pandas_df' from 'sklearn.utils.validation' (/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py)

In [41]:
import lightgbm as lgb

train_data = lgb.Dataset(X_train, label=Y_train)
model = lgb.train({}, train_data)  # Training required
preds = model.predict(X_test)
# model.score(X_test,Y_test)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000322 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 797
[LightGBM] [Info] Number of data points in the train set: 6813, number of used features: 9
[LightGBM] [Info] Start training from score 0.440775


In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import torch

# Convert tabular data to a Hugging Face Dataset
train_data = Dataset.from_pandas(X_train.assign(label=Y_train))
test_data = Dataset.from_pandas(X_test.assign(label=Y_test))

# Load a pretrained transformer (modify based on your task)
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(set(Y_train)),torch_dtype="auto")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
)

# Train the model
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<IPython.core.display.Javascript object>

In [ ]:
# Evaluate model on the test dataset
eval_results = trainer.evaluate(test_data)

# Print evaluation results
print(eval_results)
